In [ ]:
# --- Cell 0: Run PyPetaKit5D Processing (Interactive) ---
import opym
from pathlib import Path
import importlib
import sys

import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display

# Reload the opym module to make sure we have the new functions
try:
    importlib.reload(opym.petakit)
except AttributeError:
    pass
importlib.reload(opym)


# --- Create Interactive Widgets ---
print("1. Select the 'processed_tiff_series_split' directory.")
print("2. Click 'Run PetaKit Processing'")

# This chooser is set to select the processed directory
dir_chooser = FileChooser(
    path="/mmfs2/scratch/SDSMT.LOCAL/bscott/DataUpload",
    title="<b>Select Processed TIFF Directory (e.g., '.../processed_tiff_series_split'):</b>",
    show_only_dirs=True
)

run_button = widgets.Button(
    description="Run PetaKit Processing", button_style="success"
)
run_output = widgets.Output()

# --- Global variable to store the output path for the next cell ---
dsr_output_dir: Path | None = None

# --- Define the function to run on button click ---
def on_run_button_clicked(b):
    global dsr_output_dir  # <-- ADD THIS
    
    with run_output:
        run_output.clear_output()
        
        if not dir_chooser.value:
            print("❌ ERROR: No directory selected. Please select a folder and try again.")
            return

        # Use a clear variable name for the selected path
        processed_dir = Path(dir_chooser.value)
        
        # --- Store the *output* path for the viewer cell ---
        dsr_output_dir = processed_dir / "DSR"

        try:
            print(f"--- Running PetaKit5D processing for: {processed_dir.name} ---")
            
            # Call the function from your package
            opym.run_petakit_processing(
                processed_dir,
                # You can override any default parameter here, for example:
                # sheet_angle_deg=21.8,
                # interp_method="cubic"
            )
            
            print(f"\n✅ PetaKit5D job finished! Output is in: {dsr_output_dir.name}")
            print("   You can now run Cell 1 to view the results.")

        except FileNotFoundError as e:
            print(f"\n❌ ERROR: Could not find required directories or files.")
            print(f"  Details: {e}")
            print("  Please ensure you have selected the correct 'processed_tiff_series_split' folder.")
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)

# --- Link button and display UI ---
run_button.on_click(on_run_button_clicked)
display(widgets.VBox([dir_chooser, run_button, run_output]))

In [ ]:
# --- Cell 1: View Final Processed Data ---
# This cell loads and displays the final deskewed + rotated TIFF series
# from the 'DSR' directory created in Cell 0.

import opym
from pathlib import Path
import importlib
import sys

import ipywidgets as widgets
from IPython.display import display

# Reload modules
try:
    importlib.reload(opym.dataloader)
    importlib.reload(opym.viewer)
    importlib.reload(opym)
    print("Reloaded opym.dataloader and opym.viewer")
except Exception as e:
    print(f"Note: Could not reload all modules: {e}")


# --- Create Widgets ---
view_button = widgets.Button(
    description="Load & View Final DSR Data", button_style="primary"
)
view_output = widgets.Output()

# --- Define the function to run on button click ---
def on_view_button_clicked(b):
    with view_output:
        view_output.clear_output()
        
        try:
            # Check that the previous cell has been run
            if dsr_output_dir is None:
                print("❌ ERROR: 'dsr_output_dir' not set. Please run Cell 0 first.")
                return

            if not dsr_output_dir.exists():
                 print(f"❌ ERROR: DSR directory not found at: {dsr_output_dir}")
                 print("   This is unexpected. Please re-run Cell 0.")
                 return

            print(f"--- Loading data from: {dsr_output_dir.name} ---")
            
            # 1. Call the data loader
            get_stack, t_max, z_max, c_max, y, x = opym.load_tiff_series(dsr_output_dir)
            
            print(f"✅ Data loaded. Shape: T={t_max+1}, Z={z_max+1}, C={c_max+1}, Y={y}, X={x}")
            print("Displaying viewers...")
            
            # 2. Call the single channel viewer
            opym.single_channel_viewer(get_stack, t_max, z_max, c_max, y, x)
            
            # 3. Call the composite viewer
            opym.composite_viewer(get_stack, t_max, z_max, c_max, y, x)

        except FileNotFoundError as e:
            print(f"\n❌ ERROR: Could not load data.")
            print(f"  Details: {e}")
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)

# --- Link button and display UI ---
view_button.on_click(on_view_button_clicked)
display(widgets.VBox([view_button, view_output]))